In [22]:
import os
import pickle
import xlrd
import pandas as pd
from keras.models import load_model
from statsmodels.tsa.arima_model import ARIMAResults

In [23]:
import random
from tensorflow import set_random_seed
import numpy as np
seed_value =10
random.seed(seed_value)
os.environ['PYTHONHASHSEED']=str(seed_value)
np.random.seed(seed_value)
set_random_seed(seed_value)

In [3]:
def get_private_data(variables,excel_path):
    """get all the dates for x row variables"""
    workbook = xlrd.open_workbook(excel_path) #read input
    worksheet = workbook.sheet_by_index(0)

    j=9 #date column begin from 9 in format
    results ={}
    while j<worksheet.ncols:
        date3=str(xlrd.xldate.xldate_as_datetime(worksheet.cell(2,j).value,workbook.datemode))
        final_date=date3[5]+date3[6]+"-"+date3[8]+date3[9]+"-"+date3[0]+date3[1]+date3[2]+date3[3]
        ok=0
        for i in range(3,worksheet.nrows):#for each row
            if str(worksheet.cell(i,j).value)=='x':
                for var in variables:
                    if var.lower() in str(worksheet.cell(i,5).value).lower():
                        key=str(worksheet.cell(i,5).value)+"-"+str(worksheet.cell(i,3).value)
                        if key in results:
                            results[key].append(final_date)
                        else:
                            results[key]=[final_date]
        j+=1
    return results

In [4]:
def get_dataset_for_variable(variables,excel_path):
    """get dataset from the excel sheet"""
    month_convert={
            4:"Arp",
            5:"May",
            6:"Jun",
            7:"Jul",
            8:"Aug",
            9:"Sep",
            10:"Oct",
            11:"Nov",
            12:"Dec",
            1:"Jan",
            2:"Feb",
            3:"Mar"
        }
    workbook = xlrd.open_workbook(excel_path) #read input
    worksheet = workbook.sheet_by_index(0)

    keys={}
    keys["Month"]=[]

    #find all keys
    forbid={}
    for i in range(3,worksheet.nrows):#for each row
        forbid[i]=1
        for var in variables:
            if var.lower() in str(worksheet.cell(i,5).value).lower(): #column3 is product name and column 5 is variable
                key=str(worksheet.cell(i,5).value)+"-"+str(worksheet.cell(i,3).value)
                keys[key]=[]
                break

    j=9 #date column begin from 9 in format

    while j<worksheet.ncols:
        date3=str(xlrd.xldate.xldate_as_datetime(worksheet.cell(2,j).value,workbook.datemode))
        final_date=date3[5]+date3[6]+"-"+date3[8]+date3[9]+"-"+date3[0]+date3[1]+date3[2]+date3[3]
        ok=0
        for i in range(3,worksheet.nrows):#for each row
            bol=str(type(worksheet.cell(i,j).value))=="<class 'float'>"
            if str(worksheet.cell(i,j).value)=='x':
                forbid[i]=0
            if bol==1:
                for var in variables:
                    if var.lower() in str(worksheet.cell(i,5).value).lower():
                        key=str(worksheet.cell(i,5).value)+"-"+str(worksheet.cell(i,3).value)
                        keys[key].append(worksheet.cell(i,j).value)
                        ok=1
                        break
            elif forbid[i]:
                for var in variables:
                    if var.lower() in str(worksheet.cell(i,5).value).lower():
                        key=str(worksheet.cell(i,5).value)+"-"+str(worksheet.cell(i,3).value)
                        keys[key].append(0.0)
                        ok=1
                        break
        if ok==1:
            keys["Month"].append(final_date)
        j+=1

    #convert it to proper pandas format
    colname=[]
    leng=0
    for key in keys:
        colname.append(key)
        leng=len(keys[key])
    datarow=[]
    for w in range(0,leng):
        rw=[]
        for key in keys:
            rw.append(keys[key][w])
        rw.append(float(w))
        datarow.append(rw)
    colname.append('Index')
    dataset = pd.DataFrame(datarow,columns=colname).set_index('Index')
    return dataset

In [5]:
variables = ["Closing Base","Total Revenue","Net Migration","Gross Adds","Leavers"] # variables to consider for forecast
excel_path = 'PrivatizedDataforParticipants.xlsx' # excel workbook path
dataset = get_dataset_for_variable(variables,excel_path)

In [6]:
private = get_private_data(variables,excel_path)

In [7]:
dataset.head()

,Month,Closing Base-Panther,Closing Base-Leopard,Closing Base-Lion,Total Revenue-Panther - Leopard - Lion,Gross Adds(Norm)-Panther,Leavers(Norm)-Panther,Net Migrations(Norm)-Panther,Gross Adds(Norm)-Leopard,Leavers(Norm)-Leopard,Net Migrations(Norm)-Leopard,Gross Adds(Norm)-Lion,Leavers(Norm)-Lion,Net Migrations(Norm)-Lion
Index,,,,,,,,,,,,,,
0.0,04-01-2016,-2.346734,-2.043744,0.0,-1.964319,1.151483,-1.642758,1.474487,-2.121827,-1.831229,-1.517807,0.0,0.0,0.0
1.0,05-01-2016,-2.198264,-2.031028,0.0,-1.898867,0.445945,-1.425679,1.465958,-2.049060,-1.772835,-1.499497,0.0,0.0,0.0
2.0,06-01-2016,-2.044048,-1.946281,0.0,-1.864305,1.043272,-1.068171,0.511313,-0.505295,-1.741359,0.549954,0.0,0.0,0.0
3.0,07-01-2016,-1.854440,-1.760501,0.0,-1.720787,0.905359,-1.121974,0.688626,0.917019,-1.536005,0.169296,0.0,0.0,0.0
4.0,08-01-2016,-1.694420,-1.582871,0.0,-1.670489,1.138605,-1.153767,0.473906,1.658783,-1.123777,0.630261,0.0,0.0,0.0


In [8]:
def get_model(column_name):
    """load model from the directory"""
    model_path ="models"
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    model_save_path = os.path.join(model_path,column_name+".h5")
    return model_save_path

In [9]:
def get_scaler(column_name):
    """load scaler from the directory"""
    model_path ="models"
    if not os.path.exists(model_path):
        os.makedirs(model_path)
    scaler_save_path = os.path.join(model_path,'scaler-%s.sav'%column_name)
    return scaler_save_path

In [10]:
def forecast_closing_base(forecast_period = 6):
    """foreacsting closing base"""
    variables="Closing Base"
    product_name = str(input("Enter the product name(Panther/Leopard/Lion):")).title()
    column_name='%s-%s'%(variables,product_name)
    # load model
    model_save_path = get_model(column_name)
    loaded = ARIMAResults.load(model_save_path)
    forecast = loaded.get_forecast(steps=forecast_period).predicted_mean.tolist() # needed prediction
    return forecast,column_name

In [11]:
def forecast_leavers(forecast_period=6):
    """foreacsting Leavers"""
    variables="Leavers(Norm)"
    product_name = str(input("Enter the product name(Panther/Leopard/Lion):")).title()
    column_name='%s-%s'%(variables,product_name)
    # load model
    model_save_path = get_model(column_name)
    loaded = ARIMAResults.load(model_save_path)
    forecast = loaded.get_forecast(steps=forecast_period).predicted_mean.tolist() # needed prediction
    return forecast,column_name

In [12]:
def forecast_gross_adds(forecast_period=6):
    """foreacsting Gross Adds"""
    variables="Gross Adds(Norm)"
    product_name = str(input("Enter the product name(Panther/Leopard/Lion):")).title()
    column_name='%s-%s'%(variables,product_name)
    # load model
    model_save_path = get_model(column_name)
    loaded = ARIMAResults.load(model_save_path)
    forecast = loaded.get_forecast(steps=forecast_period).predicted_mean.tolist() # needed prediction
    return forecast,column_name

In [13]:
def forecast_revenue(dataset,no_of_pred = 6):
    """foreacsting Revenue"""
    forecast_period = 6
    column_name = 'Total Revenue-Panther - Leopard - Lion'
    dataset=pd.DataFrame(dataset[column_name])
    # load model
    model_save_path = get_model(column_name)
    scaler_save_path = get_scaler(column_name)
    scaler = pickle.load(open(scaler_save_path, 'rb'))
    model = load_model(model_save_path)
    testX = dataset.iloc[-no_of_pred:].values # take the last no_of_pred months value to predict next no_of_pred values
    testX = scaler.transform(np.array(testX).astype('float32').reshape(-1, 1))
    # final prediction
    final_output = []
    for i in range(0,no_of_pred):
        testX = np.reshape(testX, (1,testX.shape[0], 1))
        testPredict = model.predict(testX)  
        testX = np.reshape(testX, (testX.shape[1], 1))
        testX = np.concatenate((testX[1:],testPredict[0]))
        testPredict = np.array(testPredict[0]).flatten().reshape(-1, 1)
        output = scaler.inverse_transform(testPredict).flatten()
        output = output[0].tolist()
        final_output.append(output)
    return final_output,column_name

In [14]:
def forecast_net_migrations(dataset,no_of_pred = 6):
    """foreacsting Net Migration"""
    variables=["Net Migration"]
    product_name = str(input("Enter the product name(Panther/Leopard/Lion):")).title()
    column_name='Net Migrations(Norm)-%s'%product_name
    dataset=pd.DataFrame(dataset[column_name])
    # load model
    model_save_path = get_model(column_name)
    scaler_save_path = get_scaler(column_name)
    scaler = pickle.load(open(scaler_save_path, 'rb'))
    model = load_model(model_save_path)
    
    testX = np.array([dataset.iloc[-1]]) # take the last month value to predict next 6
    testX = scaler.transform(np.array(testX).astype('float32').reshape(-1, 1))
    final_output = []
    
    # final prediction
    for i in range(0,no_of_pred):
        testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[0]))
        testPredict = model.predict(testX)
        testX = np.array(testPredict[0])
        output = scaler.inverse_transform(testPredict)
        output = output[0].tolist()
        final_output.append(output[0])
    return final_output,column_name

In [ ]:
# uncomment each model to get the forecast for the forecast horizon
# for a different forecast horizon Revenue and Net Migrations models need retraining.

In [28]:
forecast_period = 6
# forecast_closing_base(forecast_period)
# forecast_leavers(forecast_period)
# forecast_gross_adds(forecast_period)
# forecast_revenue(dataset,forecast_period)
forecast_net_migrations(dataset,forecast_period)

Enter the product name(Panther/Leopard/Lion):lion


c:\anaconda3\envs\tagger\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator MinMaxScaler from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


([-0.32468903064727783,
  -0.24186092615127563,
  -0.176552876830101,
  -0.1249745562672615,
  -0.0841987356543541,
  -0.051944173872470856],
 'Net Migrations(Norm)-Lion')

In [16]:
## Variable keys
list(private.keys())

['Closing Base-Panther',
 'Closing Base-Leopard',
 'Closing Base-Lion',
 'Total Revenue-Panther - Leopard - Lion',
 'Gross Adds(Norm)-Panther',
 'Leavers(Norm)-Panther',
 'Net Migrations(Norm)-Panther',
 'Gross Adds(Norm)-Leopard',
 'Leavers(Norm)-Leopard',
 'Net Migrations(Norm)-Leopard',
 'Gross Adds(Norm)-Lion',
 'Leavers(Norm)-Lion',
 'Net Migrations(Norm)-Lion']

In [17]:
# private prediction dates
private['Closing Base-Panther']

['10-01-2019',
 '11-01-2019',
 '12-01-2019',
 '01-01-2020',
 '02-01-2020',
 '03-01-2020']